# Workshop 4 - cartopy


### Credit

Once again, much was adapted from the great [Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro) cartopy tutorial which, in turn, adopted much from [Phil Elson](https://pelson.github.io/)'s excellent [Cartopy Tutorial](https://github.com/SciTools/cartopy-tutorial). Phil is the creator of Cartopy and published his tutorial under an [open license](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/), meaning that we can copy, adapt, and redistribute it as long as we give proper attribution. **THANKS PHIL!** 👏👏👏

## Introduction
Making maps is often a fundamental part of geoscience and climate research. But making maps is not straightforward:

- Maps require a *projection* of geographic coordinates on the 3D Earth to the 2D space of your figure.
- Maps often include extra information besides just our data such as continents, country borders, etc.

Mapping is a notoriously hard and complicated problem, mostly due to the complexities of projection and the different possible approaches.

In this lecture, we will learn about [Cartopy](https://scitools.org.uk/cartopy/docs/latest/), one of the most common packages for making maps within Python. Another popular and powerful library is [Basemap](https://matplotlib.org/basemap/); however, Basemap is [going away](https://matplotlib.org/basemap/users/intro.html#cartopy-new-management-and-eol-announcement) and being replaced with Cartopy in the near future. For this reason, new Python learners are recommended to learn Cartopy.

If you haven't alreay done so, install cartopy with

`conda install cartopy`

The STA from last year, Janneke, had some problems with her Cartopy installation, so I will put here how she solved it. You might encounter the same problems. First, there was a problem that not all packages were updated to the correct version. If you get an error, first uninstall cartopy with

`conda uninstall cartopy`

Then update all the packages with

`conda update --all`

Then re-install cartopy again with `conda install cartopy`.

For her, it still gave problems when she wanted to use cartopy, because if you use cartopy, it will download maps from a website and one of the paths was wrong. She got a 'Download error'. This was how she solved it

She replaced line 313 in `/Users/janneke/anaconda3/lib/python3.7/site-packages/cartopy/io/shapereader.py`

    NE_URL_TEMPLATE = ('https://naciscdn.org/naturalearth/{resolution}'
                       '/{category}/ne_{resolution}_{name}.zip')
to 

    _NE_URL_TEMPLATE = ('https://naturalearth.s3.amazonaws.com/{resolution}_{category}/ne_{resolution}_{name}.zip')


I hope thas this also solves your problems! 

Now let's get started!


### Most of our media for visualization *are* flat

Two of the most common media actually are flat. Here are examples: a paper and a screen:

![](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/flat_medium.jpg)

### Projections: Taking us from spherical to flat

A map projection (or more commonly refered to as just "projection") is:

> a systematic transformation of the latitudes and longitudes of locations from the surface of a sphere or an ellipsoid into locations on a plane. [[Wikipedia: Map projection](https://en.wikipedia.org/wiki/Map_projection)].

So, in other words, we have to map a 3D sphere to a flat, 2D space.

### The major problem with map projections

The surface of a sphere is topologically different to a 2D surface, therefore we *have* to cut the sphere *somewhere*.
This means that a sphere's surface cannot be represented on a plane without distortion. See for example this orange:

![orange peel](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/orange_peel.jpg)


 
 ### Common distortions of map projections

Metric properties of maps that are often not preserved:

* Area
* Shape
* Direction
* Distance
* Scale

> all ~~models~~ map projections are wrong, but some are useful - Phileas Elson (SciPy 2018)

## Classifying projections

There are many different ways to make a projection, and we will not attempt to explain all of the choices and tradeoffs here. Instead, you can read Phil's [original tutorial](https://github.com/SciTools/cartopy-tutorial/blob/master/tutorial/projections_crs_and_terms.ipynb) for a great overview of this topic.
Instead, we will very briefly discuss a few projections and then we will dive into the more practical sides of Cartopy usage.

Two common approaches are:

 1. By [2D] surface classification
 2. By a preserving metric
 
### Projections by surface classification

![](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/projections.gif)

Downside: Not all projections can be classified in this way -> Leads to big "pseudo" and "other" groups.


### Preserving metric: Conformal

There are many metrics that can be conserved: angles, areas and distance or combinations of these two. Here, we only want to discuss briefly the conformal projection, which is also known as Orthomorphic.

These projections preserve angles locally. Implying that circles anywhere on the Earth's surface map to circles of *varying size* in the projected space.

Examples of conformal projections:

 * Mercator
 * Transverse Mercator
 * Stereographic
 * Lambert conformal conic

## Using Cartopy


For more details see the [Cartopy documentation](https://scitools.org.uk/cartopy/docs/latest/).

Cartopy makes use of a powerful library for coordinate transformations called [PROJ.4](https://proj4.org/). It also uses Numpy and shapely libraries and includes a programatic interface built on top of Matplotlib for the creation of publication quality maps.

Key features of Cartopy are its object oriented projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections.

### Cartopy Projections and other reference systems

In Cartopy, each projection is a class.
Most classes of projection can be configured in projection-specific ways, although there are some exceptions.

Let's create a Plate Carree projection instance.

To do so, we need Cartopy's crs module. This is typically imported as ``ccrs`` (Cartopy Coordinate Reference Systems).

In [ ]:
import numpy as np
import cartopy.crs as ccrs
import cartopy

Cartopy's projection list tells us that the Plate Carree projection is available with the ``ccrs.PlateCarree`` class:

https://scitools.org.uk/cartopy/docs/latest/crs/projections.html

**Note:** we need to *instantiate* the class in order to do anything projection-y with it! So we do that below:

In [ ]:
ccrs.PlateCarree()

### Drawing a map

Cartopy optionally depends upon Matplotlib, and each projection knows how to create a matplotlib Axes (or AxesSubplot) that can represent itself.

The Axes that the projection creates is a [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes). This Axes subclass overrides some of matplotlib's existing methods, and adds a number of extremely useful ones for drawing maps.

We'll go back and look at those methods shortly, but first, let's actually see the Cartopy+Matplotlib dance in action:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.axes(projection=ccrs.PlateCarree())

That was a little underwhelming, but we can see that the Axes created is indeed one of those GeoAxes[Subplot] instances.

One of the most useful methods that this class adds on top of the standard matplotlib Axes class is the ``coastlines`` method. With no arguments, it will add the Natural Earth ``1:110,000,000`` scale coastline data to the map.

In [ ]:
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

We could just as equally created a Matplotlib subplot with one of the many approaches that exist. For example, the ```plt.subplots``` function could be used:

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

Projection classes have options we can use to customize the map, we can investigate all the options using `help`

In [ ]:
help(ccrs.PlateCarree)

Using the information from above, we create a global Plate Carree map with coastlines that is centered on the dateline (180 degrees longitude), rather than the Greenwich Prime meridian.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.coastlines()

### Useful methods of a GeoAxes

The [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes) class adds a number of useful methods.

Let's take a look at:

 * [set_global](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_global) - zoom the map out as much as possible
 * [set_extent](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent) - zoom the map to the given bounding box
 

 * [gridlines](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.graticule) - add a graticule (and optionally labels) to the axes
 * [coastlines](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.coastlines) - add Natural Earth coastlines to the axes
 * [stock_img](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.stock_img) - add a low-resolution Natural Earth background image to the axes
 
 
 * [imshow](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow) - add an image (numpy array) to the axes
 * [add_geometries](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.add_geometries) - add a collection of geometries (Shapely) to the axes
 
### Some More Examples of Different Global Projections

Here, we use several of the options above!

In [ ]:
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([-170, -50, 10, 80])
ax.gridlines(draw_labels=True,
             color='black', alpha=0.2, linestyle='--')
ax.stock_img()

In the following example, we create a large figure with 6 subplots. We loop over a list of 6 different projections and add them to the figure.

In [ ]:
projections = [ccrs.PlateCarree(),
               ccrs.Robinson(),
               ccrs.Mercator(),
               ccrs.Orthographic(),
               ccrs.InterruptedGoodeHomolosine(),
               ccrs.LambertAzimuthalEqualArea()
              ]

fig = plt.figure(figsize=(15,10))
for i, proj in enumerate(projections):
    ax = fig.add_axes([(i%3)/3, .5-(i%2)/2, .33, .45], projection=proj)
    ax.stock_img()
    ax.coastlines()
    ax.set_title(f'{type(proj)}')

### Regional Maps

To create a regional map, we use the `set_extent` method of GeoAxis to limit the size of the region.

In [ ]:
help(ax.set_extent)

In [ ]:
central_lon, central_lat = -10, 45
extent = [-20, 30, 34, 70]
ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
ax.set_extent(extent)
ax.gridlines()
ax.coastlines(resolution='50m') # you can set the resolution to 10, 50 and 110m !

## Adding Features to the Map

To give our map more styles and details, we add `cartopy.feature` objects.
Many useful features are built in. These "default features" are at coarse (110m) resolution.

Name | Description
-----|------------
`cartopy.feature.BORDERS` | Country boundaries
`cartopy.feature.COASTLINE` | Coastline, including major islands
`cartopy.feature.LAKES` | Natural and artificial lakes
`cartopy.feature.LAND` | Land polygons, including major islands
`cartopy.feature.OCEAN` | Ocean polygons
`cartopy.feature.RIVERS` | Single-line drainages, including lake centerlines
`cartopy.feature.STATES` | (limited to the United States at this scale)

Below we illustrate these features in a customized map of Europe, where we add ocean, land, lakes and rivers!

In [ ]:
central_lat = 52.1
central_lon = 5.12
extent = [-20, 30, 34, 70]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)

ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS)
ax.gridlines()

If we want higher-resolution features, Cartopy can automatically download and create them from the [Natural Earth Data](http://www.naturalearthdata.com/) database or the [GSHHS dataset](https://www.ngdc.noaa.gov/mgg/shorelines/gshhs.html) database (this might take a while...).

In [ ]:
rivers_10m = cartopy.feature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m')

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)

ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(rivers_10m, facecolor='None', edgecolor='b')
ax.gridlines()

## Adding Data to the Map

Now that we know how to create a map, let's add our data to it! That's the whole point.

Because our map is a Matplotlib axis, we can use all the familiar Matplotlib commands to make plots.
By default, the map extent will be adjusted to match the data. We can override this with the `.set_global` or `.set_extent` commands.

In [ ]:
# create some test data
new_york = dict(lon=-74.0060, lat=40.7128)
honolulu = dict(lon=-157.8583, lat=21.3069)
lons = [new_york['lon'], honolulu['lon']]
lats = [new_york['lat'], honolulu['lat']]

Key point: **the data also have to be transformed to the projection space**.
This is done via the `transform=` keyword in the plotting method. The argument is another `cartopy.crs` object.
If you don't specify a transform, Cartopy assume that the data is using the same projection as the underlying GeoAxis.

From the [Cartopy Documentation](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html)

> The core concept is that the projection of your axes is independent of the coordinate system your data is defined in. The `projection` argument is used when creating plots and determines the projection of the resulting plot (i.e. what the plot looks like). The `transform` argument to plotting functions tells Cartopy what coordinate system your data are defined in.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.plot(lons, lats, label='Equirectangular straight line')
ax.plot(lons, lats, label='Great Circle', transform=ccrs.Geodetic())
ax.coastlines()
ax.legend()
ax.set_global()

### Plotting 2D (Raster) Data

The same principles apply to 2D data. Below we create some example data defined in regular lat / lon coordinates.

In [ ]:
lon = np.linspace(-80, 80, 25)
lat = np.linspace(30, 70, 25)
lon2d, lat2d = np.meshgrid(lon, lat)
data = np.cos(np.deg2rad(lat2d) * 4) + np.sin(np.deg2rad(lon2d) * 4)
plt.contourf(lon2d, lat2d, data)

Now we create a `PlateCarree` projection and plot the data on it without any `transform` keyword.
This happens to work because `PlateCarree` is the simplest projection of lat / lon data.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

However, if we try the same thing with a different projection, we get the wrong result.

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

To fix this, we need to pass the correct transform argument to `contourf`:

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data, transform=ccrs.PlateCarree())

### Showing Images

We can plot a satellite image easily on a map if we know its extent. First, download and save the following figure `https://www.visibleearth.nasa.gov/images/123103/tropical-storm-miriam-13e-off-mexico/123105l` (in the same folder as the notebook, otherwise provide the path).

Then we can plot the figure over a map. I have added a timing command, because this computation takes a while to run!

In [ ]:
%%time
# takes about a minute to transform the image
fig = plt.figure(figsize=(8, 12))

# this is from the cartopy docs
fname = 'data/Miriam.A2012270.2050.2km.jpg'
img_extent = (-120.67660000000001, -106.32104523100001,
              13.2301484511245, 30.766899999999502) #given on image website
img = plt.imread(fname)

extent = [-130,-90,10,35]
proj = ccrs.Orthographic(central_longitude=-112, central_latitude=22)
ax = plt.axes(projection=proj, extent=extent)
ax.gridlines(draw_labels=True)

# set a margin around the data
# ax.set_xmargin(0.05)
# ax.set_ymargin(0.10)

# add the image. Because this image was a tif, the "origin" of the image is in the
# upper left corner
ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
ax.coastlines(resolution='50m', color='black', linewidth=1)

# mark a known place to help us geo-locate ourselves
ax.plot(-117.1625, 32.715, 'bo', markersize=7, transform=ccrs.PlateCarree())
ax.text(-117, 33, 'San Diego', transform=ccrs.PlateCarree())

## Xarray Integration

Cartopy transforms can be passed to xarray! This creates a very quick path for creating professional looking maps from netCDF data. As an example, we will load air temperature data from the xarray-tutorial. You may need to install the library `pooch` first (it will tell you if you need that).

In [ ]:
import xarray as xr
air = xr.tutorial.open_dataset("air_temperature").air
air

First, we use `isel` from xarray, to create a new dataset that contains all temperatures at the first time step. Then we plot the data in Orthographic projection. We have set the extent to global and added coastlines! 

Note that we both use the keywords `projection` and `transform`. The `projection` argument is used when creating plots and determines the projection of the resulting plot (i.e. what the plot looks like). The `transform` argument to plotting functions tells Cartopy what coordinate system your data are defined in.

In [ ]:
p = air.isel(time=0).plot(
    subplot_kws=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"),
    transform=ccrs.PlateCarree(),
    )
p.axes.set_global() # what happens if you comment this?
p.axes.coastlines()

Here we do the same thing, but for a different projection!

In [ ]:
fig = plt.figure(figsize=(9,6))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=-90))
ax.coastlines()
ax.gridlines()
lons, lats = np.meshgrid(air.lon, air.lat)
im = ax.pcolormesh(lons, lats, air.isel(time=0)-273.15, transform=ccrs.PlateCarree())
plt.colorbar(im, ax=ax, orientation='horizontal', label=r'2m temperature [$^\circ$C]')
ax.set_global()

## Doing More

Browse the [Cartopy Gallery](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html) to learn about all the different types of data and plotting methods available!

## <span style="color:blue">Exercises</span>

1. Load the ERA5 wind data taken at different heights (heights are in hPa). 
```
ds = xr.tutorial.open_dataset(name='eraint_uvz')
```
Plot the u- and v-components at a certain height and at a certian time.
mark and label Utrecht in two projections of your choice that are not PlateCarree. Use both xarray's built-in plotting and the matplotlib approach (see above).

Use the u- and- components to create a vector plot (quiver) over `stock_img()`. Hint: use the command scale to make sure that the arrows are visibile, the scale will probablly set to abougt 400.

The figures could look like this:


![alt text](4a11.png)



![alt text](4a13.png)



2. Recreate the travel map of Phileas Fogg of Jules Verne's _Around the World in Eighty Days_ using geodetics (true shortest paths) and marking the cities as in the Wikipedia article:
![](figures/80days.png)
     1. Recreate the journey on a Robinson projection.
     2. Phileas never left the Northern Hemisphere (and thus did not truly circumnavigate the Earth), so we can plot the full journey in a projection centered around the North Pole.

Here are the places you need:
```
places = {'London': (-0.1276474, 51.5073219),
          'Suez': (32.537086, 29.974498),
          'Bombay': (72.8882172, 19.1334321),
          'Calcutta': (88.3476023, 22.5677459),
          'Hong Kong': (114.1628131, 22.2793278),
          'Yokohama': (139.636768, 35.444991),
          'San Francisco': (-122.4629897, 37.7647993),
          'New York': (-73.9866136, 40.7306458)}
```
The results could look like this:
![alt text](4a2.png)